# 15. 多线程（1）
- 参考：
    - [python开发线程:线程&守护线程&全局解释器锁](https://www.cnblogs.com/jokerbj/p/7460260.html)
    - [Understanding the Python GIL](http://www.dabeaz.com/python/UnderstandingGIL.pdf)

# 多线程 VS 多进程
- 程序：一堆代码以文本形式存入一个文档
- 进程：正在运行的程序
    - 计算机中<font color=red>**最小的资源分配单位**</font>
    - 包含地址空间、内存、数据栈等
    - 每个进程都有自己完全独立的运行环境，因此多进程共享数据是一个问题
- 线程：一个进程的独立运行片段
    - 计算机中<font color=red>**被CPU调度的最小单位**</font>，也就是说，CPU执行的其实是线程中的代码
    - 一个进程中，至少有一个线程在工作，也可以有多个线程
    - 线程相当于轻量化的进程
    - 一个进程的多个线程间共享数据及上下文运行环境
- 多个线程可以并发执行（即交替执行），但是不可以并行执行（即同时执行）
    - 因此Python中的多线程被称为“伪多线程”
    - Java、C++、C#等语言的多线程是可以并行的
    - 不能并行原因：Python解释器内部有全局解释器锁（GIL），同一时刻，统一进程中只允许有一个线程被执行，导致线程不能充分利用多核
        - 官方文档解释：在 CPython 中，由于存在 全局解释器锁，同一时刻只有一个线程可以执行 Python 代码（虽然某些性能导向的库可能会去除此限制）。 如果你想让你的应用更好地利用多核心计算机的计算资源，推荐你使用 multiprocessing 或 concurrent.futures.ProcessPoolExecutor。 但是，如果你想要同时运行多个 I/O 密集型任务，则多线程仍然是一个合适的模型。
    - 共享互斥问题
- 全局解释器锁（GIL）：
    - Python代码的执行是由Python虚拟机进行控制
    - 同一时刻，在主循环中只能有一个控制线程被执行
    - 为了防止共享资源出错，同一时刻，解释器只让一个线程访问和更改进程中的共享资源
    - GIL为了进程中数据的完整和安全，限制了程序的效率

# 实现多线程 - Python中的内置模块

## _thread：该模块有问题，不好用
- 该模块提供了操作多个线程（也被称为 轻量级进程 或 任务）的底层原语 —— 多个控制线程共享全局数据空间
- 为了处理同步问题，也提供了简单的锁机制（也称为 互斥锁 或 二进制信号）
- threading 模块基于该模块提供了更易用的高级多线程 API
- start_new_thread()：开启一个新线程并返回其标识
    - 格式：_thread.start_new_thread(function, args[, kwargs])
    - 线程执行函数 function 并附带参数列表 args (必须是元组)
    - 可选的 kwargs 参数指定一个关键字参数字典
    - 当函数返回时，线程会静默地退出
- 参考：[_thread --- 底层多线程 API — Python 3.8.2rc2 文档](https://docs.python.org/zh-cn/3/library/_thread.html?highlight=thread#module-_thread)

In [1]:
# _thread 多线程 示例1
'''
利用time函数，生成两个函数
顺序调用
计算总的运行时间
不运用多线程
顺序执行，耗时比较长
'''

import time

def loop1():
    # ctime 得到当前时间
    print('Start loop 1 at :', time.ctime())
    # 睡眠时间，单位:秒
    time.sleep(4)
    print('End loop 1 at:', time.ctime())

def loop2():
    # ctime 得到当前时间
    print('Start loop 2 at :', time.ctime())
    # 睡眠时间，单位:秒
    time.sleep(2)
    print('End loop 2 at:', time.ctime())

def main():
    print("Starting at:", time.ctime())
    loop1()
    loop2()
    print("All done at:", time.ctime())


if __name__ == '__main__':
    main()

Starting at: Wed Feb 19 17:04:14 2020
Start loop 1 at : Wed Feb 19 17:04:14 2020
End loop 1 at: Wed Feb 19 17:04:18 2020
Start loop 2 at : Wed Feb 19 17:04:18 2020
End loop 2 at: Wed Feb 19 17:04:20 2020
All done at: Wed Feb 19 17:04:20 2020


In [3]:
# _thread 多线程 示例2
'''
利用time函数，生成两个函数
顺序调用
计算总的运行时间
改用多线程，缩短总时间
'''

import time
# 导入多线程包并更名为thread
import _thread as thread


def loop1():
    # ctime 得到当前时间
    print('Start loop 1 at :', time.ctime())
    # 睡眠时间，单位:秒
    time.sleep(4)
    print('End loop 1 at:', time.ctime())


def loop2():
    # ctime 得到当前时间
    print('Start loop 2 at :', time.ctime())
    # 睡眠时间，单位:秒
    time.sleep(2)
    print('End loop 2 at:', time.ctime())


def main():
    print("Starting at:", time.ctime())
    # 启动多线程的意思是用多线程去执行某个函数
    # 启动多线程函数为start_new_thead
    # 参数两个，一个是需要运行的函数名，第二是函数的参数作为元祖使用，为空则使用空元祖
    # 注意：如果函数只有一个参数，需要参数后由一个逗号
    thread.start_new_thread(loop1, ())

    thread.start_new_thread(loop2, ())

    print("All done at:", time.ctime())


if __name__ == '__main__':
    main()
    # 创建一个死循环，使主线程可以等到子线程结束，但主线程不会关闭
    while True:
        time.sleep(1)

Starting at: Wed Feb 19 18:16:29 2020
All done at: Wed Feb 19 18:16:29 2020
Start loop 1 at : Wed Feb 19 18:16:29 2020
Start loop 2 at : Wed Feb 19 18:16:29 2020
End loop 2 at: Wed Feb 19 18:16:31 2020
End loop 1 at: Wed Feb 19 18:16:33 2020


KeyboardInterrupt: 

In [4]:
# _thread 多线程 示例3
'''
利用time延时函数，生成两个函数
利用多线程调用
计算总运行时间
练习附带参数的多线程启动方法
'''

import time
import _thread as thread

def loop1(in1):
    # ctime 得到当前时间
    print('Start loop 1 at :', time.ctime())
    # 打印参数
    print("我是参数 ", in1)
    # 睡眠时间，单位:秒
    time.sleep(4)
    print('End loop 1 at:', time.ctime())

def loop2(in1, in2):
    # ctime 得到当前时间
    print('Start loop 2 at :', time.ctime())
    # 打印参数in 和 in2，代表使用
    print("我是参数 ", in1, "和参数  ", in2)
    # 睡眠时间，单位:秒
    time.sleep(2)
    print('End loop 2 at:', time.ctime())



def main():
    print("Starting at:", time.ctime())
    # 启动多线程的意思是用多线程去执行某个函数
    # 启动多线程函数为start_new_thead
    # 参数两个，一个是需要运行的函数名，第二是函数的参数作为元祖使用，为空则使用空元祖
    # 注意：如果函数只有一个参数，需要参数后由一个逗号，这样才是元组
    thread.start_new_thread(loop1,("王老大", ))

    thread.start_new_thread(loop2,("王大鹏", "王晓鹏"))

    print("All done at:", time.ctime())

if __name__ == "__main__":
    main()
    # 一定要有while语句
    # 因为启动多线程后本程序就作为主线程存在
    # 如果主线程执行完毕，则子线程可能也需要终止
    while True:
        time.sleep(10)

Starting at: Wed Feb 19 18:34:45 2020
All done at: Wed Feb 19 18:34:45 2020
Start loop 1 at : Wed Feb 19 18:34:45 2020
我是参数  王老大
Start loop 2 at : Wed Feb 19 18:34:45 2020
我是参数  王大鹏 和参数   王晓鹏
End loop 2 at: Wed Feb 19 18:34:47 2020
End loop 1 at: Wed Feb 19 18:34:49 2020


KeyboardInterrupt: 

## threading：该模块常用
- 这个模块在较低级的模块 _thread 基础上建立较高级的线程接口
- 使用方法：直接利用 threading.Thread 生成 Thread 实例
    - 格式：threading.Thread(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
    - 调用这个构造函数时，必需带有关键字参数，但均为可选：
        - group：应该为 None；为了日后扩展 ThreadGroup 类实现而保留
        - target：是用于 run() 方法调用的可调用对象。默认是 None，表示不需要调用任何方法
        - name：是线程名称。默认情况下，由 "Thread-N" 格式构成一个唯一的名称，其中 N 是小的十进制数
        - args：是用于调用目标函数的参数元组，默认是 ()
        - kwargs：是用于调用目标函数的关键字参数字典，默认是 {}
        - daemon：参数如果是 None (默认值)，线程将继承当前线程的守护模式属性；如果不是 None，将显式地设置该线程是否为守护模式
    - 如果子类型重载了构造函数，它一定要确保在做任何事前，先发起调用基类构造器(Thread.\_\_init__())
    - 假设生成一个 Thread 实例为 t，则：
        - t.start()：开始线程活动，启动一个线程
            - 它在一个线程里最多只能被调用一次。如果同一个线程对象中调用这个方法的次数大于一次，会抛出 RuntimeError
            - 它安排对象的 run() 方法在一个独立的控制进程中调用
        - t.join(timeout=None)：等待，直到线程终结。这会阻塞调用这个方法的线程，直到被调用 join() 的线程终结（不管是正常终结还是抛出未处理异常）或者直到发生超时，超时选项是可选的
            - 当 timeout 参数存在而且不是 None 时，它应该是一个用于指定操作超时的以秒为单位的浮点数（或者分数）。因为 join() 总是返回 None ，所以你一定要在 join() 后调用 is_alive() 才能判断是否发生超时 -- 如果线程仍然存活，则 join() 超时。
            - 当 timeout 参数不存在或者是 None ，这个操作会阻塞直到线程终结。
            - 一个线程可以被 join() 很多次。
            -  如果尝试加入当前线程会导致死锁， join() 会引起 RuntimeError 异常。如果尝试 join() 一个尚未开始的线程，也会抛出相同的异常。

- 参考：[threading --- 基于线程的并行 — Python 3.8.2rc2 文档](https://docs.python.org/zh-cn/3/library/threading.html?highlight=threading#module-threading)

#### 注：
由于操作系统的不同，在Windows系统下，写多线程的时候可以不写在

<center>if __name__ == "__main__":<\center>

下面，

但是写多进程的时候一定要写在
    
<center>if __name__ == "__main__":<\center>

下面。

In [5]:
# threading 多线程 示例
'''
利用time延时函数，生成两个函数
利用多线程调用
计算总运行时间
练习带参数的多线程启动方法
'''
# 使用join()

import time
# 导入多线程处理模块
import threading

def loop1(in1):
    # ctime 得到当前时间
    print('Start loop 1 at :', time.ctime())
    # 打印参数
    print("我是参数 ",in1)
    # 睡眠时间，单位:秒
    time.sleep(4)
    print('End loop 1 at:', time.ctime())

def loop2(in1, in2):
    # ctime 得到当前时间
    print('Start loop 2 at :', time.ctime())
    # 打印参数in 和 in2，代表使用
    print("我是参数 " ,in1 , "和参数  ", in2)
    # 睡眠时间，单位:秒
    time.sleep(2)
    print('End loop 2 at:', time.ctime())


def main():
    print("Starting at:", time.ctime())
    # 生成threading.Thread实例
    t1 = threading.Thread(target=loop1, args=("王老大",))
    t1.start()

    t2 = threading.Thread(target=loop2, args=("王大鹏", "王小鹏"))
    t2.start()
    
    # 等待子线程结束
    t1.join()
    t2.join()
    
    print("All done at:", time.ctime())


if __name__ == "__main__":
    main()

Starting at: Wed Feb 19 19:41:16 2020
Start loop 1 at : Wed Feb 19 19:41:16 2020
我是参数  王老大
Start loop 2 at : Wed Feb 19 19:41:16 2020
我是参数  王大鹏 和参数   王小鹏
End loop 2 at: Wed Feb 19 19:41:18 2020
End loop 1 at: Wed Feb 19 19:41:20 2020
All done at: Wed Feb 19 19:41:20 2020


### 守护线程
- daemon：一个表示这个线程是（True）否（False）守护线程的布尔值
- 设置方法1：t.setDaemon(True)
- 设置方法2：t.daemon = True
    - 一定要在调用 start() 前设置好，不然会抛出 RuntimeError
    - 初始值继承于创建线程；主线程不是守护线程，因此主线程创建的所有线程默认都是 daemon = False
    - 当没有存活的非守护线程时，整个Python程序才会退出
- 守护进程和守护线程的区别：
    - 守护进程守护的是主进程，即主进程**代码执行完毕（不代表进程结束）**之后就会结束
    - 守护线程守护的是所有的子线程，即所有的**非守护线程执行完毕**之后才会结束
- 在程序中，如果将子线程设置成守护线程，则该子线程会在主线程结束时自动退出
- 主线程结束结束时，主进程随之结束
- 守护线程是否有效与运行环境有关
- 一般，守护线程不重要或不允许离开主线程独立运行

In [3]:
# 非守护线程 示例
import time
import threading

def func():
    print("Start func")
    time.sleep(3)
    print("end func") # 因为主线程先结束，所以该句最后打印

print("Main thread")

t1 = threading.Thread(target=func, args=() )
t1.start()

time.sleep(1)
print("Main thread end")

Main thread
Start func
Main thread end
end func


In [5]:
# 守护线程 示例
import time
import threading

def func():
    print("Start func")
    time.sleep(3)
    print("end func")

print("Main thread")

t1 = threading.Thread(target=func, args=() )
# 设置守护线程，必须在start()之前设置，否则无效
# 方法一
t1.setDaemon(True)
# 方法二
#t1.daemon = True
t1.start()

time.sleep(1)
print("Main thread end")

# 因为Jupyter的环境原因，导致守护线程无效
# PyCharm 中将不会再打印 “end func”

Main thread
Start func
Main thread end
end func


### 线程常用函数
- threading.current_thread()：返回当前对应调用者的控制线程的 Thread 对象
    - 如果调用者的控制线程不是利用 threading 创建，会返回一个功能受限的虚拟线程对象
- threading.currentThread()：返回当前的线程对象
    - - threading.currentThread().ident：返回当前线程的ID
- threading.enumerate()：以列表形式返回当前所有存活的 Thread 对象
    - 该列表包含守护线程，current_thread() 创建的虚拟线程对象和主线程
    - 不包含已终结的线程和尚未开始的线程
- threading.active_count()：返回当前存活的线程类 Thread 对象
    - 返回的计数等于 enumerate() 返回的列表长度，效果跟 len(threading.enumerate)相同
- thr.setName()：给线程设置名字，直接当做特征属性使用它（thr 为 Thread 对象名）
- thr.getName()：得到线程的名字，直接当做特征属性使用它（thr 为 Thread 对象名）
- thr.is_alive()：判断线程是否在活动（thr 为 Thread 对象名）

In [7]:
from threading import Thread, currentThread

def func():
    print('子线程：', currentThread(), t.ident)  # 子线程可以直接用 Thread 类里的 ident 属性获取

t = Thread(target=func)
t.start()
print('主线程：', currentThread(), currentThread().ident)  # 主线程的 ID 要用 currentThread().ident 获取

子线程： <Thread(Thread-12, started 17516)> 17516
主线程： <_MainThread(MainThread, started 9380)> 9380


In [19]:
# 常用函数 示例
import time
import threading

def loop1():
    # ctime 得到当前时间
    print('Start loop 1 at :', time.ctime())
    # 睡眠时间，单位:秒
    time.sleep(6)
    print('End loop 1 at:', time.ctime())

def loop2():
    # ctime 得到当前时间
    print('Start loop 2 at :', time.ctime())
    # 睡眠时间，单位:秒
    time.sleep(1)
    print('End loop 2 at:', time.ctime())

def loop3():
    # ctime 得到当前时间
    print('Start loop 3 at :', time.ctime())
    # 睡眠时间，单位:秒
    time.sleep(5)
    print('End loop 3 at:', time.ctime())


def main():
    print("Starting at:", time.ctime())
    
    # 生成threading.Thread实例
    t1 = threading.Thread(target=loop1, args=( ))
    # setName是给子线程设置一个名字
    t1.setName("THR_1")
    t1.start()
    
    # 生成threading.Thread实例
    t2 = threading.Thread(target=loop2, args=( ))
    t2.setName("THR_2")
    t2.start()
    
    # 生成threading.Thread实例
    t3 = threading.Thread(target=loop3, args=( ))
    t3.setName("THR_3")
    t3.start()

    # 预期4秒后，thread2已经自动结束，
    time.sleep(4)
    
    # enumerate 得到正在运行子线程，即子线程1和子线程3
    for thr in threading.enumerate():
        # getName能够得到线程的名字
        print("正在运行的线程名字是： {0}".format(thr.getName()))

    print("正在运行的子线程数量为： {0}".format(threading.activeCount()))
    
    # 等待子线程结束
    t1.join()
    t2.join()
    t3.join()
    
    print("All done at:", time.ctime())


if __name__ == "__main__":
    main()

Starting at: Wed Feb 19 22:00:46 2020
Start loop 1 at : Wed Feb 19 22:00:46 2020
Start loop 2 at : Wed Feb 19 22:00:46 2020
Start loop 3 at : Wed Feb 19 22:00:46 2020
End loop 2 at: Wed Feb 19 22:00:47 2020
正在运行的线程名字是： MainThread
正在运行的线程名字是： Thread-4
正在运行的线程名字是： Thread-5
正在运行的线程名字是： IPythonHistorySavingThread
正在运行的线程名字是： Thread-3
正在运行的线程名字是： THR_1
正在运行的线程名字是： THR_3
正在运行的子线程数量为： 7
End loop 3 at: Wed Feb 19 22:00:51 2020
End loop 1 at: Wed Feb 19 22:00:52 2020
All done at: Wed Feb 19 22:00:52 2020


### 生成Thread实例的另一种方法——直接继承自 threading.Thread
- 直接继承 Thread
- 需重写 run() 函数
- 类实例可直接运行

In [20]:
# 直接继承自 threading.Thread 示例
import time
import threading

# 1. 类需要继承自threading.Thread
class MyThread(threading.Thread):
    def __init__(self, arg):
        super(MyThread, self).__init__()
        self.arg = arg

    # 2 必须重写run函数，run函数代表的是真正执行的功能
    def  run(self):
        time.sleep(2)
        print("The args for this class is {0}".format(self.arg))

for i in range(5):
    t = MyThread(i)
    t.start()
    t.join()

print("Main thread is done!!!!!!!!")

The args for this class is 0
The args for this class is 1
The args for this class is 2
The args for this class is 3
The args for this class is 4
Main thread is done!!!!!!!!


In [21]:
# 企业常用多线程实例
from time import sleep, ctime
import threading

loop = [4,2]

class ThreadFunc:

    def __init__(self, name):
        self.name = name

    def loop(self, nloop, nsec):
        '''
        :param nloop: loop函数的名称
        :param nsec: 系统休眠时间
        :return:
        '''
        print('Start loop ', nloop, 'at ', ctime())
        sleep(nsec)
        print('Done loop ', nloop, ' at ', ctime())

def main():
    print("Starting at: ", ctime())

    # ThreadFunc("loop").loop 跟一下两个式子相等：
    # t = ThreadFunc("loop")
    # t.loop
    # 以下 t1 和  t2 的定义方式相等
    t = ThreadFunc("loop")
    t1 = threading.Thread( target = t.loop, args=("LOOP1", 4))
    # 下面这种写法更西方人，工业化一点
    t2 = threading.Thread( target = ThreadFunc('loop').loop, args=("LOOP2", 2))

    # 常见错误写法
    #t1 = threading.Thread(target=ThreadFunc('loop').loop(100,4))
    #t2 = threading.Thread(target=ThreadFunc('loop').loop(100,2))

    t1.start()
    t2.start()

    t1.join()
    t2.join()


    print("ALL done at: ", ctime())


if __name__ == '__main__':
    main()

Starting at:  Wed Feb 19 22:12:04 2020
Start loop  LOOP1 at  Wed Feb 19 22:12:04 2020
Start loop  LOOP2 at  Wed Feb 19 22:12:04 2020
Done loop  LOOP2  at  Wed Feb 19 22:12:06 2020
Done loop  LOOP1  at  Wed Feb 19 22:12:08 2020
ALL done at:  Wed Feb 19 22:12:08 2020
